## 分类图像

In [83]:
import torch
import torchvision
from torch.utils.data import DataLoader
# 准备数据集
train_data = torchvision.datasets.CIFAR10(root='data', train=True, transform=torchvision.transforms.ToTensor(),download=True)
test_data = torchvision.datasets.CIFAR10(root='data', train=False, transform=torchvision.transforms.ToTensor(),download=True)

print("训练集的长度:{}".format(len(train_data)))
print("测试集的长度:{}".format(len(test_data)))

Files already downloaded and verified
Files already downloaded and verified
训练集的长度:50000
测试集的长度:10000


In [61]:
# DataLoader加载数据集
train_dataloader = DataLoader(train_data, batch_size=8)
test_dataloader = DataLoader(test_data, batch_size=8)

x, y = next(iter(train_dataloader))
len(train_dataloader)
x.shape, y

(torch.Size([8, 3, 32, 32]), tensor([6, 9, 9, 4, 1, 1, 2, 7]))

In [78]:
# 定义cnn模型
class Model(torch.nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        
        #520的卷积层
        self.cnn1 = torch.nn.Conv2d(in_channels=3,
                                    out_channels=16,
                                    kernel_size=5,
                                    stride=2,
                                    padding=0)
        #311的卷积层
        self.cnn2 = torch.nn.Conv2d(in_channels=16,
                                    out_channels=32,
                                    kernel_size=3,
                                    stride=1,
                                    padding=1)
        #710的卷积层
        self.cnn3 = torch.nn.Conv2d(in_channels=32,
                                    out_channels=128,
                                    kernel_size=7,
                                    stride=1,
                                    padding=0)
        #池化层
        self.pool = torch.nn.MaxPool2d(kernel_size=2,stride=2)
        
        #激活函数
        self.relu = torch.nn.ReLU()
        
        #全连接层
        self.fc = torch.nn.Linear(in_features=128,out_features=10)
        
            #定义模型计算部分
    def forward(self,x):
        
        # 第一次，卷积变换可以推演
        #[8,3,32,32] -> [8,16,14,14]
        x = self.cnn1(x)
        x = self.relu(x)
        
        # 第二次,因为是311的卷积，所有尺寸不变
        #[8,16,14,14] -> [8,32,14,14]
        x = self.cnn2(x)
        x = self.relu(x)
        
        #池化，尺寸变为一半
        #[8,32,14,14] -> [8,32,7,7]
        x = self.pool(x)
        
        # 第三次，710的卷积，尺寸变为3
        #[8,32,7,7] -> [8,128,1,1]
        x = self.cnn3(x)
        x = self.relu(x)
        
        #将数据展平,便于线性运算，也先当于把图像变成向量
        #[8,128,1,1] -> [8,128]
        x = x.flatten(start_dim=1)
        
        #线性运算
        #[8,128] -> [8,10]
        return self.fc(x)
    
model = Model()

model(torch.randn(8,3,32,32)).shape

torch.Size([8, 10])

In [80]:
#训练模型
def train():
    optimizier = torch.optim.Adam(model.parameters(),lr=1e-3)
    loss_func = torch.nn.CrossEntropyLoss()
    model.train()
    
    for epoch in range(5):
        for i, (x,y) in enumerate(train_dataloader):
            out = model(x)
            loss = loss_func(out,y)
            
            loss.backward()
            optimizier.step()
            optimizier.zero_grad()
            
            if i % 1000 == 0:
                acc = (out.argmax(dim=1) == y).sum().item() / len(y)
                print("epoch:{}, step:{}, loss:{}, acc:{}".format(epoch, i, loss.item(), acc))
            
        torch.save(model, "model2.model")
        
train()

epoch:0, step:0, loss:2.305931806564331, acc:0.125
epoch:0, step:1000, loss:1.5132529735565186, acc:0.125
epoch:0, step:2000, loss:1.7923462390899658, acc:0.375
epoch:0, step:3000, loss:1.3619712591171265, acc:0.5
epoch:0, step:4000, loss:1.3302316665649414, acc:0.5
epoch:0, step:5000, loss:0.9700559377670288, acc:0.75
epoch:0, step:6000, loss:2.20335054397583, acc:0.375
epoch:1, step:0, loss:0.6032549738883972, acc:0.875
epoch:1, step:1000, loss:0.9141734838485718, acc:0.75
epoch:1, step:2000, loss:1.3747743368148804, acc:0.75
epoch:1, step:3000, loss:1.1171748638153076, acc:0.5
epoch:1, step:4000, loss:1.032057762145996, acc:0.75
epoch:1, step:5000, loss:0.935653567314148, acc:0.75
epoch:1, step:6000, loss:1.7614184617996216, acc:0.375
epoch:2, step:0, loss:0.3983888030052185, acc:0.875
epoch:2, step:1000, loss:0.8888397216796875, acc:0.875
epoch:2, step:2000, loss:1.4221217632293701, acc:0.625
epoch:2, step:3000, loss:0.8549391627311707, acc:0.5
epoch:2, step:4000, loss:0.9626393318

In [82]:
#测试模型
@torch.no_grad()
def test():
    model = torch.load("model2.model")
    model.eval()
    
    total_num = 0
    total_correct = 0
    
    for x,y in test_dataloader:
        out = model(x)
        pred = out.argmax(dim=1)
        total_correct += (pred == y).sum().item()
        total_num += len(y)
        
    print("acc:{}".format(total_correct / total_num))

test()

acc:0.6352
